In [20]:
from cellitaire.environment.agents.PPOAgent import Agent
from cellitaire.environment.cellitaire_env import CellitaireEnv
from cellitaire.environment.rewards.reward import *
from cellitaire.environment.rewards.foundation_rewards import *
import numpy as np

board_rows = 7
board_cols = 12
num_reserved = 6
test_reward = CombinedReward([
    PlacedCardInFoundationReward(weight=6),
    WinReward(weight=100, rows=board_rows, cols=board_cols),
    #ConstantReward(weight=0.5),
    #ScalingPlacedCardInFoundationReward(weight=1, rows=board_rows, cols=board_cols),
    #PlayedLegalMoveReward(weight=1, rows=board_rows, cols=board_cols, num_reserved = num_reserved),
    #PeriodicPlacedCardInFoundationReward(weight=4, reward_period=3),
    #CreatedMovesReward(weight=1, num_reserved=num_reserved, foundation_count_dropoff=30)
])
env = CellitaireEnv(test_reward, rows=board_rows, cols=board_cols, num_reserved=num_reserved, max_moves=1200, max_illegal_moves=1200)
#env.render()

#agent = Agent(n_actions=env.action_space.n, batch_size=batch_size, 
#                alpha=alpha, n_epochs=n_epochs, 
#                input_dims=env.observation_space.shape)

N = 10000
batch_size = 2000
n_epochs = 5
alpha = 1e-8
agent = Agent(n_actions=env.action_space.n, 
    input_dims=(board_rows * board_cols * 4 + 6,), batch_size=batch_size, 
                alpha=alpha, n_epochs=n_epochs)

In [21]:
env.action_space.n = board_rows * board_cols

In [22]:
agent.load_models()
n_games = 300

best_score = -1000
score_history = []
max_score = 0
episodes_without_best = 0

learn_iters = 0
avg_score = 0
n_steps = 0
i = 0
while True:
    env.reset()
    observation = env.get_state()
    done = False
    truncated = False
    score = 0
    while not done and not truncated:
        action, prob, val = agent.choose_legal_action_mostly(observation, env.get_legal_actions_as_int())
        observation_, reward, done, truncated, info = env.step(action)
        n_steps += 1
        score += reward
        agent.remember(observation, action, prob, val, reward, done)
        if n_steps % N == 0:
            agent.learn()
            learn_iters += 1
        observation = observation_
    i += 1
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    recent_max = np.max(score_history[-110:])
    
    max_score = max(max_score, score)
    
    if avg_score > best_score and n_steps > N:
        best_score = avg_score
        recent_std = np.std(score_history[-100:])
        #agent.save_models()
        episodes_without_best = 0
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max Rec {recent_max:>5.1f} | max AT {max_score:>5.1f} | ls {learn_iters:>5} | done {done} *')
    else:
        episodes_without_best += 1
        
    if episodes_without_best % 100 == 0 and episodes_without_best > 0:
        recent_std = np.std(score_history[-100:])
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max Rec {recent_max:>5.1f} | max AT {max_score:>5.1f} | ls {learn_iters:>5} | done {done}')


... loading models ...
episode    21 | score  204.0 | avg  166.6 | std  67.30 | max Rec 264.0 | max AT 264.0 | ls     1 | done True *
episode    22 | score  264.0 | avg  171.0 | std  68.81 | max Rec 264.0 | max AT 264.0 | ls     1 | done True *
episode    24 | score  192.0 | avg  171.5 | std  66.04 | max Rec 264.0 | max AT 264.0 | ls     1 | done True *
episode    26 | score  234.0 | avg  172.2 | std  65.24 | max Rec 264.0 | max AT 264.0 | ls     1 | done True *
episode    28 | score  270.0 | avg  174.6 | std  65.69 | max Rec 270.0 | max AT 270.0 | ls     1 | done True *
episode   128 | score  180.0 | avg  164.6 | std  75.83 | max Rec 288.0 | max AT 288.0 | ls     5 | done True
episode   140 | score  180.0 | avg  174.8 | std  74.31 | max Rec 288.0 | max AT 288.0 | ls     6 | done True *
episode   144 | score  276.0 | avg  175.9 | std  75.56 | max Rec 288.0 | max AT 288.0 | ls     6 | done True *
episode   145 | score  264.0 | avg  176.3 | std  75.93 | max Rec 288.0 | max AT 288.0 | ls 

KeyboardInterrupt: 

In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make('CartPole-v1', render_mode='human')
N = 20
batch_size = 5
n_epochs = 4
alpha = 0.0003
agent = Agent(n_actions=env.action_space.n, batch_size=batch_size, 
                alpha=alpha, n_epochs=n_epochs, 
                input_dims=env.observation_space.shape)
n_games = 300

figure_file = 'plots/cartpole.png'

best_score = -1000
score_history = []
max_score = 0
episodes_without_best = 0


learn_iters = 0
avg_score = 0
n_steps = 0

for i in range(n_games):
    observation = env.reset()[0]
    done = False
    truncated = False
    score = 0
    while not done and not truncated:
        action, prob, val = agent.choose_action(observation)
        observation_, reward, done, truncated, info = env.step(action)
        n_steps += 1
        score += reward
        agent.remember(observation, action, prob, val, reward, done)
        if n_steps % N == 0:
            agent.learn()
            learn_iters += 1
        observation = observation_
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    
    max_score = max(max_score, score)
    
    if avg_score > best_score:
        best_score = avg_score
        recent_std = np.std(score_history[-100:])
        #agent.save_models()
        episodes_without_best = 0
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done} *')
    else:
        episodes_without_best += 1
        
    if episodes_without_best % 100 == 0 and episodes_without_best > 0:
        recent_std = np.std(score_history[-100:])
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done}')


In [ ]:
env.close()

In [23]:
agent.save_models()

... saving models ...


In [ ]:
demo_env.close()

In [ ]:
%pip install -e ../.